In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from gensim.models import KeyedVectors

# Item embedding: TF-IDF matrix

In [5]:
news_train = 'RecDatasets/conversion_tools/MINDsmall_dev/news.tsv'
news_test = 'RecDatasets/conversion_tools/MINDsmall_dev/news.tsv'


def load_df(path):
    if 'news' in path:
        columns = ['News ID',
                "Category",
                "SubCategory",
                "Title",
                "Abstract",
                "URL",
                "Title Entities",
                "Abstract Entities"]

    elif 'behavior' in path:
        columns = ['Impression ID',
                "User ID",
                "Time",
                "History",
                "Impressions"]
    else:
        return pd.read_csv(path, sep='\t', header=None)

    df = pd.read_csv(path, sep='\t', header=None, names=columns)
    return df
news_train, news_test = load_df(news_train), load_df(news_test)
data = pd.concat([news_train, news_test])
inter = pd.read_csv('/Users/giulia/Desktop/tesi/mind_small15/mind_small15.inter', sep='\t', header=0)
#inter = inter[inter['label:float'] != 0] # keep only clicked articles

In [6]:
#remove the N in front of the news ID
data['News ID'] = data['News ID'].str[1:]
#drop everything but news ID & title
col = ['News ID', 'Title']
data = data[col]

In [7]:
#make the news ID string
data['News ID'] = data['News ID'].astype(str)
inter['item_id:token'] = inter['item_id:token'].astype(str)
unique_newsid = data['News ID'].unique()
unique_itemid = inter['item_id:token'].unique()

In [8]:
#check common news ID
common = np.intersect1d(unique_newsid, unique_itemid)
print(len(common), len(unique_newsid), len(unique_itemid))
#keep in data only the one in common
data = data[data['News ID'].isin(common)]
print(data.shape)
data = data.drop_duplicates(subset='News ID') #since it is made of both train and test
print(data.shape)

21695 42416 26937
(43390, 2)
(21695, 2)


In [ ]:
#preprocessing
english_stopwords = set(stopwords.words('english'))
stemmer = PorterStemmer()

pattern_punctuation = re.compile(r'[^\w\s]')
pattern_numbers = re.compile(r'\w*\d+\w*')
pattern_short_words = re.compile(r'\b\w{1,3}\b')

def preprocess_text(text):
    text = text.lower()
    text = pattern_punctuation.sub('', text)  # del punctuation
    text = pattern_numbers.sub('', text)  # del numbers
    text = pattern_short_words.sub('', text)  # del words with len <= 2
    words = text.split()
    words = [word for word in words if word not in english_stopwords]  # del stopwords
    words = [stemmer.stem(word) for word in words]  # stemming
    return ' '.join(words)

data['processed_title'] = data['Title'].apply(preprocess_text)

[TfidfVectorizer documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
#tfidf embedding
tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.95, stop_words='english', max_features=3000, norm='l2') #since DMF uses cos similarity L2 seems appropriate normalization
tfidf_matrix = tfidf_vectorizer.fit_transform(data['processed_title'])
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
tfidf_df['News ID'] = data['News ID'].values

In [ ]:
tfidf_df.reset_index(drop=True, inplace=True)
#put column NEWS ID as first column
tfidf_df = tfidf_df[ ['News ID'] + [ col for col in tfidf_df.columns if col != 'News ID' ] ]
tfidf_df
tfidf_df.to_csv('tfidf_emb.csv', index=False)

,News ID,aaron,abandon,abl,abort,absolut,absurd,abus,accept,access,...,youth,youtub,youv,yovanovitch,zealand,zion,zodiac,zone,zozo,zuckerberg
0,61837,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,53526,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,38324,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,2073,0.0,0.0,0.566778,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,49186,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25227,5072,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
25228,31080,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
25229,62355,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.934059,0.0,0.0,0.0
25230,63860,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


# User Embedding from Item Embedding

1. aggregate  news + average:   $$ u = \frac{1}{n} \sum_{i=1}^{n} v_i $$ <br>
where  $n$ is the number of items interacted with by the user, and $v_i$​ is the TF-IDF vector of the i-th item <br><br><br>
2. aggregate  news + weighted average based on label: $$ u = \frac{\sum_{i=1}^{n} w_i v_i}{\sum_{i=1}^{n} w_i }  $$ <br>
   e.g. $w_i = 0.1$ for item seen and not interacted and $w_i = 1$ otherwise <br>
   <br><br>
   e.g. a weight of 0.1 for item seen and not interacted (i.e. label = 0) scales down the contribution of this vector to the overall profile by 90%. <br><b>Q: Does this approach make sense with very sparse data?-> Maybe viable if embedding was denser e.g. word2vec or glove</b>

In [3]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

#tfidf = tfidf_df
tfidf = pd.read_csv('/content/drive/MyDrive/tesi/data/tfidf_emb.csv')
inter = pd.read_csv('/content/drive/MyDrive/tesi/data/mind_small15.inter', sep='\t', header=0)
inter = inter[inter['label:float'] != 0]
assert inter['item_id:token'].nunique() == tfidf.shape[0]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:

tfidf_columns = tfidf.columns[:-1]
#initialize user vector & counter
user_tfidf_sum = pd.DataFrame(0, index=inter['user_id:token'].unique(), columns=tfidf_columns)
user_interaction_counts = pd.Series(0, index=inter['user_id:token'].unique())

#due to suicidal kernel, let's try incremental approach
chunk_size = 10000

for start in range(0, inter.shape[0], chunk_size):
    end = min(start + chunk_size, inter.shape[0])
    chunk = inter.iloc[start:end]

    user_news_chunk = pd.merge(chunk, tfidf, left_on='item_id:token', right_on='News ID')
    #sum of interacted item (emb)
    user_sum = user_news_chunk.groupby('user_id:token')[tfidf_columns].sum()
    user_tfidf_sum.loc[user_sum.index] += user_sum
    #count inter in chnk
    user_counts = user_news_chunk['user_id:token'].value_counts()
    user_interaction_counts[user_counts.index] += user_counts

#average over seen items
user_embeddings = user_tfidf_sum.div(user_interaction_counts, axis=0)


In [3]:
user_embeddings.rename(columns={'Unnamed: 0': 'uid:token'}, inplace=True)
user_embeddings

,uid:token,aaron,abandon,abl,abort,absolut,absurd,abus,accept,access,...,youth,youtub,youv,yovanovitch,zealand,zion,zodiac,zone,zozo,zuckerberg
0,13740,0.0,0.0,0.0,0.043779,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,91836,0.0,0.0,0.0,0.000000,0.0,0.0,0.006283,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,73700,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,34670,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,8125,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86331,73518,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
86332,16981,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
86333,17300,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
86334,57759,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
#prepare and save as atomic file
user_embeddings = user_embeddings.set_index('uid:token')
user_embeddings['user_emb:float_seq'] = user_embeddings.apply(lambda row: ' '.join(row.astype(str)), axis=1)
user_embeddings.reset_index(inplace=True)
user_embeddings = user_embeddings[['uid:token', 'user_emb:float_seq']]
user_embeddings.to_csv('user_embeddings15.csv', index=False)


In [7]:
user_embeddings

,uid:token,user_emb:float_seq
0,13740,0.0 0.0 0.0 0.0437792077908793 0.0 0.0 0.0 0.0...
1,91836,0.0 0.0 0.0 0.0 0.0 0.0 0.0062834390437269 0.0...
2,73700,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
3,34670,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
4,8125,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
...,...,...
86331,73518,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
86332,16981,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
86333,17300,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
86334,57759,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....


In [2]:
#same for item embedding
item_emb = pd.read_csv('/Users/giulia/Desktop/tesi/tfidf_emb.csv', sep=',')

In [ ]:
item_emb = item_emb.set_index('News ID')
#change name of columns
item_emb.rename(columns={'News ID': 'iid:token'}, inplace=True)
item_emb['item_emb:float_seq'] = item_emb.apply(lambda row: ' '.join(row.astype(str)), axis=1)
item_emb.reset_index(inplace=True)
item_emb = item_emb[['News ID', 'item_emb:float_seq']]


In [15]:
#item_emb.rename(columns={'News ID': 'iid:token', 'item_emb:float_seq': 'item_emb:float_seq'}, inplace=True)
item_emb

,iid:token,item_emb:float_seq
0,61837,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
1,53526,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
2,38324,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
3,2073,0.0 0.0 0.5667783692028184 0.0 0.0 0.0 0.0 0.0...
4,49186,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
...,...,...
25227,5072,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
25228,31080,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
25229,62355,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
25230,63860,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....


In [16]:
item_emb.to_csv('item_embeddings15.csv', sep='\t', index=False)

# Pre-trained embeddings: <br>
average of [GloVe](https://nlp.stanford.edu/projects/glove/) embedding weighted by tf-idf of words in the article

In [20]:
#why don't they already put this will forever be a mistery to me
def add_header_to_glove_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    num_vectors = len(lines)
    dimensions = 200
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(f"{num_vectors} {dimensions}\n")
        file.writelines(lines)
input_file = 'glove.twitter.27B/glove.twitter.27B.200d.txt'
output_file = 'glove.twitter.27B/glove.twitter.27B.200d.txt'
add_header_to_glove_file(input_file, output_file)


In [21]:
glove_model = KeyedVectors.load_word2vec_format('glove.twitter.27B/glove.twitter.27B.200d.txt', binary=False)

In [22]:
#preprocessing
english_stopwords = set(stopwords.words('english'))
stemmer = PorterStemmer()

pattern_punctuation = re.compile(r'[^\w\s]')
pattern_numbers = re.compile(r'\w*\d+\w*')
pattern_short_words = re.compile(r'\b\w{1,3}\b')

def glove_preprocess_text(text):
    text = text.lower()
    text = pattern_punctuation.sub('', text)  # del punctuation
    text = pattern_numbers.sub('', text)  # del numbers
    text = pattern_short_words.sub('', text)  # del words with len <= 2
    words = text.split()
    words = [word for word in words if word not in english_stopwords]  # del stopwords
    #words = [stemmer.stem(word) for word in words]  #GloVe uses exact form of words
    return words

data['processed_title'] = data['Title'].apply(glove_preprocess_text)

In [23]:
tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.95, stop_words='english', norm='l2', tokenizer=lambda x: x, preprocessor=lambda x: x)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['processed_title'])
feature_names = tfidf_vectorizer.get_feature_names_out()

/Users/giulia/miniconda3/envs/recbole-env/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/giulia/miniconda3/envs/recbole-env/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn(


In [24]:
#GloVe + TF-IDF Embeddings
def get_weighted_average_glove(words, tfidf_vector, tfidf_feature_names):
    word_vectors = np.zeros((glove_model.vector_size,))
    total_weight = 0
    for word in words:
        if word in glove_model and word in tfidf_feature_names:
            tfidf_weight = tfidf_vector[0, tfidf_feature_names.tolist().index(word)]
            word_vectors += glove_model[word] * tfidf_weight #weighted sum
            total_weight += tfidf_weight
    if total_weight > 0:
        word_vectors /= total_weight #averaged
    return word_vectors #200D vector

In [ ]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
glove_embeddings = np.vstack([get_weighted_average_glove(doc, tfidf_matrix[i], tfidf_feature_names) for i, doc in enumerate(data['processed_title'])])
data['glove_embeddings'] = list(glove_embeddings)

In [ ]:
#ITEM EMBEDDING as atomic file
item_emb = data[['News ID', 'glove_embeddings']]
item_emb.rename(columns={'News ID': 'iid:token', 'glove_embeddings': 'item_emb:float_seq'}, inplace=True)

In [47]:
item_emb['item_emb:float_seq'] = item_emb['item_emb:float_seq'].apply(np.array)
item_emb.to_csv('item_embeddings15_glove.csv', sep='\t', index=False)
item_emb

/var/folders/c8/lq7gcths36b2vmqfhj9vyn8r0000gn/T/ipykernel_35293/987449394.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_emb['item_emb:float_seq'] = item_emb['item_emb:float_seq'].apply(np.array)


,iid:token,item_emb:float_seq
2,61837,"[-0.30197723697031764, -0.014468840086915978, ..."
3,53526,"[0.03010406329930957, -0.15387444922323448, -0..."
4,38324,"[-0.1454820249636669, -0.5390467918998917, -0...."
5,2073,"[0.018984402547169994, 0.23328780069890165, -0..."
7,49186,"[-0.2725303832914561, 0.19907965545186296, -0...."
...,...,...
42409,42491,"[-0.36948703839288705, -0.0434063660035074, -0..."
42410,13097,"[0.17317195598627697, 0.22560175342431157, -0...."
42411,63550,"[-0.08543534452090845, 0.00893242611954955, 0...."
42412,30345,"[0.24341000616550446, 0.2530199885368347, -0.5..."


In [ ]:
#USER EMBEDDING